In [1]:
import copy
from data_loading import utils, purged_group_time_series
from models import resnet, lightning_nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch.nn as nn
from torch.utils.data import DataLoader
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pytorch_lightning as pl
import numpy as np
import torch
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import lightgbm as lgb
import optuna
import joblib
from models.SupervisedAutoEncoder import SupAE, create_hidden_rep
import os

In [2]:
p_ae = {'dim_1': 675, 'dim_2': 400, 'dim_3': 224, 'hidden': 162,
         'activation': nn.ReLU, 'dropout': 0.2916447561918717, 'lr': 0.030272591341587315,
         'recon_loss_factor': 0.4447516076774931, 'batch_size': 1252, 'loss_sup_ae': nn.MSELoss,
         'loss_recon': nn.MSELoss,
         'embedding': True,
         'input_size':310,'output_size':1}

#p_res = {'dim_1': 843, 'dim_2': 2637, 'dim_3': 1618, 'dim_4': 880, 'dim_5': 220, 'activation': nn.LeakyReLU, 'dropout': 0.453246718032545, 'lr': 0.04565788979670108, 'batch_size': 10836,'loss':nn.MSELoss,'embedding':True,'input_size':310,'output_size':1,'hidden_len':p_ae['hidden']}
#model_res = resnet.ResNet(input_size=p_res['input_size'],output_size=p_res['output_size'],params=p_res)
#model_res.load_state_dict(torch.load('./saved_models/trained/ResNet_state_dict.pth'))
model_ae = utils.load_model(path='./saved_models/trained/trained_ae.pth',p=p_ae,pl_lightning=False,model=SupAE)
models_dict = {'ResNet':[model_res,p_res],'ae':[model_ae,p_ae]}


NameError: name 'model_res' is not defined

In [3]:
#utils.create_predictions(models=models_dict)

In [4]:
utils.create_prediction_file()

,id,prediction
0,n002a15bc5575bbb,0.494021
1,n00309caaa0f955e,0.494062
2,n00576b397182463,0.494071
3,n00633405d59c6a1,0.494200
4,n008c2eefc8911c7,0.494225
...,...,...
5034,nffd9a4732077912,0.493923
5035,nffe5dd1fd84013b,0.494102
5036,nffeea35c1a65c79,0.494011
5037,nffefe9565943594,0.494187


In [3]:
pred_path = f"{utils.get_data_path(root_dir='./data')}/predictions/"

In [15]:
df = utils.load_data(root_dir='./data/',mode='test')

In [5]:
val_eras = df[df['data_type']=='validation']['era'].unique().tolist()

In [6]:
eras = os.listdir(pred_path)

In [7]:
eras = [f"era{era.replace('.csv','')}" for era in eras]

In [8]:
eras = [era for era in eras if era in val_eras]

In [9]:
eras =[era.replace('era','') for era in eras]

In [10]:
eras = [f'{era}.csv' for era in eras]

In [31]:
df_preds = utils.create_prediction_file()

In [3]:
def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]


# convenience method for scoring
def scoring(df):
    return correlation(df['prediction'], df['target'])


# Payout is just the score cliped at +/-25%
def payout(scores):
    return scores.clip(lower=-0.25, upper=0.25)

In [20]:
df = df.merge(df_preds, on = 'id')

In [30]:
scores = df.groupby('era').apply(scoring)
np.mean(scores)

-0.0051975484992210575

In [32]:
df_preds

,id,prediction
0,n002a15bc5575bbb,0.494021
1,n00309caaa0f955e,0.494062
2,n00576b397182463,0.494071
3,n00633405d59c6a1,0.494200
4,n008c2eefc8911c7,0.494225
...,...,...
5034,nffd9a4732077912,0.493923
5035,nffe5dd1fd84013b,0.494102
5036,nffeea35c1a65c79,0.494011
5037,nffefe9565943594,0.494187


In [2]:
df = utils.load_data(root_dir='./data/',mode='test')
df = df[df['data_type'] == 'validation']


In [3]:
df_test = df

In [4]:
df = utils.load_data(root_dir='./data/',mode='train')

In [7]:
df['era'][df['era'] == 'eraX'] = 'era999'

<ipython-input-7-cd23c978c5ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['era'][df['era'] == 'eraX'] = 'era999'


In [5]:
data,target,features,era = utils.preprocess_data(df,ordinal=True)
data_test,target_test,features_test,era_test = utils.preprocess_data(df_test,ordinal=True)
"""
data_ = copy.deepcopy(data)
target_ = copy.deepcopy(target)
oe = OrdinalEncoder()
data = oe.fit_transform(data)
target=oe.fit_transform(target_.values.reshape(-1,1)).reshape(-1)
"""

'\ndata_ = copy.deepcopy(data)\ntarget_ = copy.deepcopy(target)\noe = OrdinalEncoder()\ndata = oe.fit_transform(data)\ntarget=oe.fit_transform(target_.values.reshape(-1,1)).reshape(-1)\n'

In [6]:
del df ,df_test

In [7]:
## Autoencodr training
data = np.concatenate([data,data_test],0)

In [8]:
target = np.concatenate([target,target_test],0)
era = np.concatenate([era,era_test],0)

In [9]:
#t_idx = np.arange(start=0,stop=len(era),step=1).tolist()
t_idx =np.where(era <121)[0].tolist()
v_idx =np.where(era >=121)[0].tolist()


In [13]:
p = joblib.load('./hpo/params/SupAEnn_hpo_2021-04-05.pkl')

In [14]:
p = p.best_params

In [15]:
p['activation'] = nn.ReLU
p['loss_sup_ae']= nn.MSELoss
p['loss_recon']= nn.MSELoss
p['embedding']=True

In [16]:
dataset = utils.FinData(data = data,target=target,era=era)
dataloaders = utils.create_dataloaders(dataset,indexes={'train':t_idx,'val':v_idx},batch_size=p['batch_size'])
p['input_size'] = len(features)
p['output_size'] = 1
model = SupAE(params=p)
es = EarlyStopping(monitor='val_sup_loss',patience=10,min_delta=0.0005,mode='min')
trainer = pl.Trainer(max_epochs=100,gpus=1,callbacks=[es])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [19]:
batch = next(iter(dataloaders['val']))
batch['era']

tensor([[121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121., 121.,
         121., 121., 121., 1

In [18]:
trainer.fit(model,train_dataloader = dataloaders['train'],val_dataloaders=dataloaders['val'])
torch.save(model.state_dict(),f'./saved_models/trained/ae_state_dict.pth')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | loss_recon       | MSELoss    | 0     
1 | loss_sup_ae      | MSELoss    | 0     
2 | embedding_layers | ModuleList | 4.7 K 
3 | encoder          | Sequential | 1.3 M 
4 | regressor        | Sequential | 232   
5 | decoder          | Sequential | 1.3 M 
------------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
10.174    Total estimated model params size (MB)
/home/james/.virtualenvs/Kaggle/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


Epoch 0:  79%|███████▊  | 807/1028 [00:53<00:14, 15.15it/s, loss=1.11, v_num=19, val_loss=1.070, val_sup_loss=0.201, train_loss=1.110]
Validating: 0it [00:00, ?it/s]
Epoch 1:  15%|█▌        | 155/1028 [00:10<00:57, 15.07it/s, loss=1.11, v_num=19, val_loss=0.990, val_sup_loss=0.0502, train_loss=1.110, sup_loss=0.0531, recon_loss=1.210] 


In [12]:
p_xgb = joblib.load('./hpo/params/xgb_hpo_2021-04-05.pkl').best_params
p_lgb = joblib.load('./hpo/params/lgb_hpo_2021-04-05.pkl').best_params
p_lgb['boosting']= 'gbdt'
p_lgb['max_depth']= 50
p_lgb['objective']= 'regression'
p_lgb['metric']= 'mse'
p_lgb['n_jobs']=-1

In [13]:
data_dict = data_dict = {'data': data, 'target': target,
                                 'features': features, 'era': era}

In [14]:
x_tr, x_val = data_dict['data'][t_idx], data_dict['data'][v_idx]
y_tr, y_val = data_dict['target'][t_idx],data_dict['target'][v_idx]
d_tr = lgb.Dataset(x_tr,label=y_tr)
d_val = lgb.Dataset(x_val,label=y_val)
clf = lgb.train(p_lgb,d_tr,1000,valid_sets=[d_val],early_stopping_rounds=50,verbose_eval=True)
clf.save_model(f'./saved_models/trained/lgb.bin')

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026486 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1550
[LightGBM] [Info] Number of data points in the train set: 501808, number of used features: 310
[LightGBM] [Info] Start training from score 0.499997
[1]	valid_0's l2: 0.0499474
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 0.049947
[3]	valid_0's l2: 0.0499466
[4]	valid_0's l2: 0.0499462
[5]	valid_0's l2: 0.0499458
[6]	valid_0's l2: 0.0499456
[7]	valid_0's l2: 0.0499454
[8]	valid_0's l2: 0.0499451
[9]	valid_0's l2: 0.0499445
[10]	valid_0's l2: 0.049944
[11]	valid_0's l2: 0.0499438
[12]	valid_0's l2: 0.0499434
[13]	valid_0's l2: 0.0499432
[14]	valid_0's l2: 0.049943
[15]	valid_0's l2: 0.0499427
[16]	valid_0's l2: 0.0499423
[17]	valid_0's l2: 0.0499421
[18]	valid_0's l2: 0.0499417
[19]	valid_0's l2:

In [15]:
data_dict = {'data': data, 'target': target,
                 'features': features, 'era': era}

In [18]:
p_ae['input_size'] = len(features)
p_ae['output_size'] = 1
model = utils.load_model('./saved_models/trained/trained_ae.pth',
                    p=p_ae, pl_lightning=False, model=SupAE)


In [16]:

models_dict = {'lgb':[clf,p_lgb]}

In [19]:
ae_output = create_hidden_rep(model, data_dict)
data_dict['hidden'],ae_preds = ae_output['hidden'], ae_output['preds']
data_dict['hidden_true'] = True
p['hidden_len'] = data_dict['hidden'].shape[-1]

NameError: name 'p' is not defined

In [ ]:
p

In [16]:
dataset = utils.FinData(
            data=data_dict['data'], target=data_dict['target'], era=data_dict['era'], hidden=data_dict.get('hidden', None))
dataloaders = utils.create_dataloaders(
            dataset, indexes={'train': t_idx}, batch_size=p['batch_size'])

In [7]:
p_res = {'dim_1': 843, 'dim_2': 2637, 'dim_3': 1618, 'dim_4': 880, 'dim_5': 220, 'activation': nn.LeakyReLU, 'dropout': 0.453246718032545, 'lr': 0.04565788979670108, 'batch_size': 10836,'loss':nn.MSELoss,'embedding':True,'input_size':310,'output_size':1,'hidden_len':p_ae['hidden']}

In [8]:
model = resnet.ResNet(input_size = p['input_size'],output_size=p['output_size'],params=p)
es = EarlyStopping(monitor='val_mse',patience=10,min_delta=0.0005,mode='min')
trainer = pl.Trainer(max_epochs=100,gpus=1,callbacks=[es])
trainer.fit(model,train_dataloader=dataloaders['train'],val_dataloaders=dataloaders['val'])
torch.save(model.state_dict(),f'./saved_models/trained/ResNet_state_dict.pth')

NameError: name 'p' is not defined

In [18]:
#model = utils.load_model('./saved_models/trained/ResNet_state_dict.pth',
 #                   p=p, pl_lightning=False, model=resnet.ResNet)


<All keys matched successfully>

In [9]:
model_res = resnet.ResNet(input_size=p_res['input_size'],output_size=p_res['output_size'],params=p_res)
model_res.load_state_dict(torch.load('./saved_models/trained/ResNet_state_dict.pth'))
model_ae = utils.load_model(path='./saved_models/trained/trained_ae.pth',p=p_ae,pl_lightning=False,model=SupAE)
models_dict = {'ResNet':[model_res,p_res],'ae':[model_ae,p_ae]}


In [18]:
utils.create_predictions(models=models_dict)

FileNotFoundError: [Errno 2] No such file or directory: './data/numerai_dataset_258/predictions/132.csv'

NameError: name 'predictions' is not defined

In [16]:
p['input_size'] = len(features)
p['output_size'] = 1
train = True
if train:
    gts = purged_group_time_series.PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=5)
    models=[]
    for i, (train_idx,val_idx) in enumerate(gts.split(data,groups=era)):
        dataset = utils.FinData(data=data, target=target, era=era)
        dataloaders = utils.create_dataloaders(
        dataset, indexes={'train': train_idx, 'val': val_idx}, batch_size=p['batch_size'])
        model = SupAE(p)
        es = EarlyStopping(monitor='val_loss', patience=10,
                            min_delta=0.005, mode='min')
        trainer = pl.Trainer(max_epochs=100,
                                gpus=1,
                                callbacks=[es])
        trainer.fit(
            model, train_dataloader=dataloaders['train'], val_dataloaders=dataloaders['val'])
        torch.save(model.state_dict(), f'saved_models/ae_fold_{i}_state_dict.pth')
        models.append(model)
else:
    models_nn = utils.load_model('./saved_models/AE',p=p,pl_lightning=False,model=SupAE)


2, train_loss=1.090]
Epoch 1:  42%|████▏     | 118/283 [00:08<00:11, 14.24it/s, loss=1.09, v_num=20, val_loss=0.972, val_sup_loss=0.0511, train_loss=1.090, sup_loss=0.0748, recon_loss=1.190]
Validating: 0it [00:00, ?it/s]
Epoch 1:  49%|████▉     | 139/283 [00:09<00:09, 15.09it/s, loss=1.09, v_num=20, val_loss=0.972, val_sup_loss=0.0511, train_loss=1.090, sup_loss=0.0748, recon_loss=1.190]
/home/james/.virtualenvs/Kaggle/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | loss_recon       | MSELoss    | 0     
1 | loss_sup_ae      | MSELoss    | 0     
2 | embedding_layers | ModuleList | 4.7 K 
3 | encoder          | Sequential | 1.3 M 
4 | regressor  

In [ ]:
data_,_,_,_ = utils.preprocess_data(df,nn=True)

p = {'learning_rate': 0.030803514080008098,
         'max_leaves': 50,
         'bagging_fraction': 0.9011886437667906,
         'bagging_freq': 5,
         'feature_fraction': 0.3287921216266973,
         'min_data_in_leaf': 396,
         'lambda_l1': 0.02217696438630042,
         'lambda_l2': 0.03985756503899372,
         'boosting': 'gbdt',
         'max_depth': 50,
         'objective': 'regression',
         'metric': 'mse',
         'n_jobs':-1}
gts = purged_group_time_series.PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=5)
models_gbm = []
scores = []
for i, (tr_idx,val_idx) in enumerate(gts.split(data_,groups=era)):
    x_tr, x_val = data_[tr_idx], data_[val_idx]
    y_tr, y_val = target[tr_idx],target[val_idx]
    d_tr = lgb.Dataset(x_tr,label=y_tr)
    d_val = lgb.Dataset(x_val,label=y_val)
    clf = lgb.train(p,d_tr,1000,valid_sets=[d_val],early_stopping_rounds=50,verbose_eval=True)
    preds = clf.predict(x_val)
    score = mean_squared_error(y_val,preds)
    print(f'Fold {i} MSE score:\t', score)
    models_gbm.append(clf)
    scores.append(score)

In [ ]:
del data_,df,data

In [ ]:
input_size = data
output_size= 1
p = {'dim_1': 1500,
    'dim_2': 1000,
    'dim_3': 500,
    'dim_4': 250,
    'dim_5': 150,
    'activation': nn.SiLU,
    'dropout': 0.2,
    'lr': 0.05,
    'label_smoothing':0.1,
    'amsgrad': False,
    'batch_size': 5000,
    'loss':nn.MSELoss(),
    'embedding':True}

In [ ]:
dataset = utils.FinData(data=data, target=target, era=era)
del df

In [ ]:
model = models_nn[-1]
model.eval().to('cuda')
index = np.linspace(0,dataset.data.shape[0],dataset.data.shape[0],dtype='int')

In [ ]:
batch_size = 5000
dataloaders = utils.create_dataloaders(dataset,{'train':index.tolist()},batch_size=batch_size)

In [ ]:
hiddens =[]
for i, batch in enumerate(dataloaders['train']):
    _,hidden,_,_ = model(batch['data'].view(batch['data'].size(1),-1).to('cuda'))
    hiddens.append(hidden.cpu().detach().numpy())

In [ ]:
hiddens = np.array([hiddens[i][j] for i in range(len(hiddens)) for j in range(len(hiddens[i]))])

In [ ]:
data  = np.concatenate([data,hiddens],axis=1)

In [ ]:
gts = purged_group_time_series.PurgedGroupTimeSeriesSplit(n_splits=5,group_gap=5)
dataset = utils.FinData(data=data, target=target, era=era)
models=[]
for i, (train_idx,val_idx) in enumerate(gts.split(data,groups=era)):
    
    dataloaders = utils.create_dataloaders(
    dataset, indexes={'train': train_idx, 'val': val_idx}, batch_size=p['batch_size'])
    model = resnet.ResNet(input_size=input_size,output_size=output_size,params=p)
    #model.apply(lambda x: utils.init_weights(x,'relu'))
    es = EarlyStopping(monitor='val_mse', patience=10,
                        min_delta=0.005, mode='min')
    trainer = pl.Trainer(max_epochs=100,
                            gpus=1,
                            callbacks=[es])
    trainer.fit(
        model, train_dataloader=dataloaders['train'], val_dataloaders=dataloaders['val'])
    torch.save(model.state_dict(), f'saved_models/Resnet/fold_{i}_state_dict.pth')
    models.append(model)



In [ ]:
df = utils.load_data(root_dir='./data/',mode='test')

In [ ]:
data,target,features,era = utils.preprocess_data(df.iloc[:10000,],test=True,ordinal=True)
data_,_,_,_=utils.preprocess_data(df.iloc[:10000,],test=True,nn=True)
"""
data = data[0:1000]
oe = OrdinalEncoder()
data = oe.fit_transform(data)
"""

In [ ]:
t = torch.tensor()

In [ ]:
data = torch.LongTensor(data)

In [ ]:
data.shape

In [ ]:
def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]


# convenience method for scoring
def scoring(df):
    return correlation(df['preds'], df['target'])


# Payout is just the score cliped at +/-25%
def payout(scores):
    return scores.clip(lower=-0.25, upper=0.25)

In [ ]:
preds = []
for model in models_nn:
    model.eval()
    preds.append(model(data))


In [ ]:
predictions = [preds[i][1].detach().numpy() for i in range(len(preds))]

In [ ]:
predictions = np.mean(predictions,axis=0)

In [ ]:
predictions = predictions.reshape(-1)

In [ ]:
df_preds = pd.DataFrame.from_dict({'era':era,'preds':predictions,'target':target.T})


In [ ]:
corrs = df_preds.groupby('era').apply(scoring)
corrs

In [ ]:
payout(corrs).mean()


In [ ]:
predictions = models_nn[0](data).reshape(-1).detach().numpy()

In [ ]:
preds = []
for model in models_gbm:
    preds.append(model.predict(data_))

In [ ]:
predictions_gbm = np.mean(preds,axis=0)
df_gbm = pd.DataFrame.from_dict({'era':era,'preds':predictions_gbm,'target':target.T})

In [ ]:
corrs_gbm = df_gbm.groupby('era').apply(scoring)
corrs_gbm

In [ ]:
payout(corrs_gbm).mean()

In [ ]:
df[df['data_type']=='test']

In [ ]:

wojhed-riDni0-hopnok